In [1]:
import random,math,copy,time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier)
from sklearn import tree
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from tqdm import tqdm
import time

C:\Users\SONAL BANSAL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
m=2186  # Problem dimension (number of decision variables
Ap=0.1  # Awareness probability
fl=2   # Flight length (fl)
wf=1.2# wf is used to control the importance of classification accuracy and number of selected features.
d=67600# Flock (population) size
lt=67600

In [3]:
ft=pd.read_csv("ft.csv")
ft=ft.drop(columns=['Unnamed: 0'])
ft=pd.Series(ft['0'])


In [4]:
imo.to_csv('importance_feature_AD-MCI.csv')
imo=imo.drop(columns=['Unnamed: 0'])
imo=pd.Series(imo['0'])


In [5]:
features=[i for i in range(0,lt)]
features[-1]

67599

In [6]:
def init():   #initialization function
    df2=pd.DataFrame(np.ones((m,d),dtype='int'),columns=features)
    return df2

In [10]:
# this defines the fitness of each crow by calculating their accuracy and no features selected using the below formula
def fitness(df2):      # fitness function whose 1 parameter takes the crows and their position as input
    fnt=np.zeros((m,1))  # fnt is the fitness function
    for k in tqdm(range(m)):    
        global sel
        sel=[]
        q=0
        imp=0
        a=(df2.iloc[k,:])==0.0
        if a.all()==True:
            df2.iloc[k,:]=1.0
        for p in features:
            if df2.loc[k,p]==1.0:
                sel.insert(q,p)
                temp=imo[p]
                imp=imp+temp
            q=q+1
        lf=len(sel)
        divide=lf/lt
        fnt[k]=imp+wf*(1-float(lf/lt))     # fitness formula
        del sel
    return fnt


In [9]:
z=init()   #crow memory with initial position
tmax=1 #   # Maximum number of iterations (itermax)
xn=z.copy()  # the position of the crow
mem=init()  # memory is initialized using the init function
xnew=(xn).copy()   # the positions of the crow
ft=fitness(xn)
plotit=[]


In [ ]:
tottime=0
s=pd.DataFrame(z)
print("got s")    # it is a formula to convert positions of each crow into 0 and 1 
for t in range(tmax):
    print(t)# no of iterations
    time1=time.time()
   
    xnew_list = []
    plotit.insert(t,ft.max())
    num = random.randint(0,m-1)  
   # Generation of random candidate crows for following (chasing)
    for i in tqdm(range(m)):   
       
        r=random.random()
        if r>=Ap:
            
            xnew_list.append((xn.loc[i,:]+fl*r*(mem.loc[num,:]-xn.loc[i,:])).tolist())# Generation of a new position for crow i (state 1)
            
        else:
            
            xnew_list.append((xnew.loc[i,:].apply(lambda lam:random.random())).tolist())   # Generation of a new position for crow i (state 2)
            
            
    xnew = pd.DataFrame(xnew_list)
    del xnew_list
    print('transpose')
    xnew= 1/( 1 + np.expm1(( 10*(xnew-0.5 ) ) ))    #  formula to convert positions of each crow into 0 and 1
    print('exp')
    xnew=xnew.apply(lambda l:l>=random.random()).astype(int)
    print('apply lambda')
    xn=xnew.copy()     
    print('copied')
    # it is the updated binary position of each crow
    #feasibility test
    # this is done so that to check if atleast one position of the crow has value equal to 1. if it is not true then every
    # position value of the crow is made 1.
    for i in tqdm(range(m)):
        a=(xn.loc[i,:])==0.0
        if a.all()==True:
            xn.loc[i,:]=1.0
    print('running fitness')
    ft_new=fitness(xn).reshape(ft.shape)# here the accuracy of each crows are calculated.

    print('fitness done')
    for i in tqdm(range(m)):
        
        if ft_new[i]>ft[i]:
            ft[i]=ft_new[i]
            for j in features:
                mem.loc[i,j]= xn.loc[i,j]
    print('complete')
    time2=time.time()
    tottime=tottime+(time2-time1)
print(tottime/50)

got s
0


100%|██████████████████████████████████████████████████████████████████████████████| 2186/2186 [00:47<00:00, 45.63it/s]


transpose
exp
apply lambda
copied


100%|█████████████████████████████████████████████████████████████████████████████| 2186/2186 [00:21<00:00, 103.81it/s]


running fitness


100%|████████████████████████████████████████████████████████████████████████████| 2186/2186 [2:04:12<00:00,  3.41s/it]


fitness done


  3%|█▉                                                                         | 57/2186 [1:33:35<58:15:32, 98.51s/it]

In [ ]:
# here the accuracy of each crows are calculated.

neigh = KNeighborsClassifier(n_neighbors=3)
rforest = RandomForestClassifier()
dtree = tree.DecisionTreeClassifier()
clf = svm.SVC()

nacc1=np.zeros((m,1),dtype='float64')
rfacc1=np.zeros((m,1),dtype='float64')
dtacc1=np.zeros((m,1),dtype='float64')
clf1=np.zeros((m,1),dtype='float64')

q=0
for i in tqdm(range(m)):
    print(i)
    select=[]
    for p in list(features):
        if mem.loc[i,p]==1:
            select.insert(q,p)
            q=q+1
    neigh.fit(x_train[select], y_train)
    neigh.predict(x_test[select])
    nacc1[i]=neigh.score(x_test[select],y_test) 

    
    rforest.fit(x_train[select], y_train)
    rforest.predict(x_test[select])
    rfacc1[i]=rforest.score(x_test[select],y_test) 

    dtree.fit(x_train[select], y_train)
    dtree.predict(x_test[select])
    dtacc1[i]=dtree.score(x_test[select],y_test)

select=[]
for p in list(features):
    if mem.loc[i,p]==1:
        select.insert(q,p)
        q=q+1

print (nacc1.max(), rfacc1.max(), dtacc1.max()) # the crow with maximum accuracy is printed